In [ ]:
%load_ext autoreload
%autoreload 2


In [1]:
import pandas as pd

# importa desde tu paquete (el nombre debe coincidir con tu __init__.py)
from Isea import ScatterBrushOld   # si tu paquete se llama distinto, ajusta aquí

# lee tu CSV local
df = pd.read_csv("Isea/datos_vis.csv")

# lanza el scatter con brush (área)
ScatterBrushOld(df, x="horsepower", y="weight", color="origin")


<IPython.core.display.Javascript object>

In [ ]:
from IPython.display import HTML, Javascript, display
display(HTML("<div id='ok' style='padding:6px;border:1px solid #ddd'>Esperando JS…</div>"))
display(Javascript("document.getElementById('ok').innerText = 'JS OK ✅';"))


In [ ]:
from IPython.display import Javascript, display
display(Javascript("""
(async () => {
  const mod = await import('https://cdn.jsdelivr.net/npm/d3@7/+esm');
  const d3v = mod.default ?? mod;
  document.getElementById('ok').innerText = 'D3 cargado ✅ (keys: ' + Object.keys(d3v).length + ')';
})();
"""))


In [ ]:
import pandas as pd
from Isea import RadarChart

radar_data = pd.DataFrame([
    {"name": "Jugador A", "velocidad": 80, "fuerza": 90, "resistencia": 70, "técnica": 85, "precisión": 75},
    {"name": "Jugador B", "velocidad": 95, "fuerza": 60, "resistencia": 85, "técnica": 70, "precisión": 90}
])

RadarChart(
    df=radar_data,
    features=["velocidad", "fuerza", "resistencia", "técnica", "precisión"],
    width=700,
    height=700
)

In [ ]:
import pandas as pd
from Isea.graphs import ScatterBrush, RadarChart, Sunburst, StreamGraph

# Cargar datos
df = pd.read_csv("Isea/datos_vis.csv")
print(f"Datos cargados: {len(df)} filas")
df.head()

In [ ]:
df_sunburst = df.copy()

# Crear categoría de peso
df_sunburst['weight_category'] = pd.cut(
    df_sunburst['weight'], 
    bins=3, 
    labels=['Ligero', 'Medio', 'Pesado']
)

Sunburst(
    df=df_sunburst,
    path_columns=['origin', 'cylinders', 'weight_category'],
    value_column='horsepower',
    width=800,
    height=800,
    color_scheme='schemeSet3'
)

In [ ]:
df_stream = df.groupby(['year', 'origin'])['horsepower'].mean().reset_index()

StreamGraph(
    df=df_stream,
    x_column='year',
    y_column='horsepower',
    category_column='origin',
    width=900,
    height=500,
    color_scheme='schemeTableau10'
)

In [ ]:
df_sunburst2 = df[['cylinders', 'origin', 'mpg']].copy()
df_sunburst2['cylinders'] = df_sunburst2['cylinders'].astype(str) + ' cyl'

Sunburst(
    df=df_sunburst2,
    path_columns=['cylinders', 'origin'],
    value_column='mpg',
    width=700,
    height=700,
    color_scheme='schemePastel1'
)

In [ ]:
df_stream2 = df.copy()
df_stream2['cylinders_cat'] = df_stream2['cylinders'].astype(str) + ' cyl'
df_stream2 = df_stream2.groupby(['model_year', 'cylinders_cat'])['acceleration'].mean().reset_index()

StreamGraph(
    df=df_stream2,
    x_column='year',
    y_column='acceleration',
    category_column='cylinders_cat',
    width=950,
    height=550,
    color_scheme='schemeCategory10'
)

In [ ]:
radar_data = df.groupby('origin')[['horsepower', 'weight', 'acceleration', 'mpg']].mean().reset_index()
radar_data.columns = ['name', 'caballos', 'peso', 'aceleracion', 'eficiencia']

RadarChart(
    df=radar_data,
    features=['caballos', 'peso', 'aceleracion', 'eficiencia'],
    name_col='name',
    width=700,
    height=700
)

In [4]:
# Celda 1: Cargar datos y aplicar PCA para mejores visualizaciones
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from Isea.graphs import ScatterBrush, RadarChart, Sunburst, StreamGraph

# Cargar datos
df = pd.read_csv("Isea/datos_vis.csv")

# Limpiar datos (eliminar filas con valores faltantes en horsepower)
df = df.dropna(subset=['horsepower'])

print(f"Datos cargados: {len(df)} filas")
print(f"Columnas: {df.columns.tolist()}")

# Aplicar PCA a las características numéricas
numeric_cols = ['mpg', 'displacement', 'horsepower', 'weight', 'acceleration']
X = df[numeric_cols].dropna()

# Estandarizar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA a 2 componentes
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Crear DataFrame con componentes PCA
df_pca = df.loc[X.index].copy()
df_pca['PC1'] = X_pca[:, 0]
df_pca['PC2'] = X_pca[:, 1]

print(f"\nVarianza explicada por PCA:")
print(f"PC1: {pca.explained_variance_ratio_[0]:.2%}")
print(f"PC2: {pca.explained_variance_ratio_[1]:.2%}")
print(f"Total: {sum(pca.explained_variance_ratio_):.2%}")

df_pca.head()

ModuleNotFoundError: No module named 'sklearn'

In [3]:
# Celda 2: SCATTER con PCA - Visualización de componentes principales
ScatterBrushOld(
    df=df_pca, 
    x="PC1", 
    y="PC2", 
    color="origin",
    width=900,
    height=600
)

NameError: name 'df_pca' is not defined

In [ ]:
# 1) Prepara datos (año, pivot por país y estándar de tecnologías)
import pandas as pd

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]
for c in year_cols: df[c] = pd.to_numeric(df[c], errors="coerce")

IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
f = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()
YEAR = [c for c in year_cols if f[c].notna().any()][-1]

tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
f["Technology_std"] = f["Technology"].map(tech_map).fillna(f["Technology"])

# Pivot por país (1 punto = país) para el scatter
idx = pd.MultiIndex.from_product([f["Country"].dropna().unique(),
                                  sorted(f["Technology_std"].dropna().unique())],
                                 names=["Country","Technology_std"])
grid = (f.groupby(["Country","Technology_std"])[YEAR].sum(min_count=1)
          .reindex(idx, fill_value=0.0).reset_index())
piv = (grid.pivot(index="Country", columns="Technology_std", values=YEAR)
          .fillna(0.0).reset_index())

for c in ["Solar","Wind","Hydro","Bio","Fossil"]:
    if c not in piv.columns: piv[c] = 0.0
piv["Total"] = piv.drop(columns=["Country"]).sum(axis=1, numeric_only=True)
piv["DominantTech"] = piv[["Solar","Wind","Hydro","Bio","Fossil"]].idxmax(axis=1)

# Para que no se amontonen, quitamos (0,0) solo del SCATTER:
piv_viz = piv[(piv["Solar"]>0) | (piv["Wind"]>0)].copy()
print(f"Año: {YEAR} | países con Solar/Wind>0: {len(piv_viz)}  | filas totales CSV: {len(f)}")


In [ ]:
from Isea import ParallelEnergy

years = [c for c in f.columns if c.startswith("F") and f[c].notna().any()]

ParallelEnergy(
    df=f,                       # tabla larga con múltiples años
    years=years,                # ["F2000",...,"F2023"]
    tech_col="Technology_std",
    label_col="Country",
    dims=("Solar","Wind","Hydro","Bio","Fossil"),  # orden inicial
    year_start="F2023",         # año inicial en el slider
    width=1150, height=600,
    log_axes=False,             # True si quieres escalas log
    normalize=False,            # True para comparar “forma” (0-1)
    reorder=True                # puedes arrastrar ejes
)


In [ ]:
# 2) Scatter (1 punto = país) con tooltip completo y leyenda clicable
from Isea import ScatterBrush

ScatterBrush(
    piv_viz,
    x="Solar", y="Wind",
    color="DominantTech",
    size="Total",
    label="Country",
    zero_filter=False,
    log_x=False, log_y=False,
    width=960, height=560
)


In [ ]:
# 3) BeeSwarm (USA TODAS LAS FILAS del año: 1 punto = país–tecnología)
from Isea import BeeSwarmCapacity
BeeSwarmCapacity(f, year_col=YEAR, tech_col="Technology_std", country_col="Country",
                 width=960, height=560, log_y=True)


Graphs for expose

first do this

In [2]:
# 1) Prepara datos (año, pivot por país y estándar de tecnologías)
import pandas as pd

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]
for c in year_cols: df[c] = pd.to_numeric(df[c], errors="coerce")

IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
f = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()
YEAR = [c for c in year_cols if f[c].notna().any()][-1]

tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
f["Technology_std"] = f["Technology"].map(tech_map).fillna(f["Technology"])

# Pivot por país (1 punto = país) para el scatter
idx = pd.MultiIndex.from_product([f["Country"].dropna().unique(),
                                  sorted(f["Technology_std"].dropna().unique())],
                                 names=["Country","Technology_std"])
grid = (f.groupby(["Country","Technology_std"])[YEAR].sum(min_count=1)
          .reindex(idx, fill_value=0.0).reset_index())
piv = (grid.pivot(index="Country", columns="Technology_std", values=YEAR)
          .fillna(0.0).reset_index())

for c in ["Solar","Wind","Hydro","Bio","Fossil"]:
    if c not in piv.columns: piv[c] = 0.0
piv["Total"] = piv.drop(columns=["Country"]).sum(axis=1, numeric_only=True)
piv["DominantTech"] = piv[["Solar","Wind","Hydro","Bio","Fossil"]].idxmax(axis=1)

# Para que no se amontonen, quitamos (0,0) solo del SCATTER:
piv_viz = piv[(piv["Solar"]>0) | (piv["Wind"]>0)].copy()
print(f"Año: {YEAR} | países con Solar/Wind>0: {len(piv_viz)}  | filas totales CSV: {len(f)}")


Año: F2023 | países con Solar/Wind>0: 82  | filas totales CSV: 570


In [ ]:
from Isea import ParallelEnergy

years = [c for c in f.columns if c.startswith("F") and f[c].notna().any()]

ParallelEnergy(
    df=f,                       # tabla larga con múltiples años
    years=years,                # ["F2000",...,"F2023"]
    tech_col="Technology_std",
    label_col="Country",
    dims=("Solar","Wind","Hydro","Bio","Fossil"),  # orden inicial
    year_start="F2023",         # año inicial en el slider
    width=1150, height=600,
    log_axes=False,             # True si quieres escalas log
    normalize=False,            # True para comparar “forma” (0-1)
    reorder=True                # puedes arrastrar ejes
)


In [ ]:
import pandas as pd
from Isea import WorldRenewable

df = pd.read_csv("Isea/Energy_clean.csv")

# Filtra a capacidad instalada en MW
df = df[(df["Indicator"] == "Electricity Installed Capacity") &
        (df["Unit"] == "Megawatt (MW)")].copy()

# Mapea Technology -> Technology_std (5 tecnologías)
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
if "Technology_std" not in df.columns:
    df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

# Columnas de año
year_cols = [c for c in df.columns if isinstance(c, str) and c.startswith("F") and df[c].notna().any()]
start = "F2023" if "F2023" in year_cols else year_cols[-1]

WorldRenewable(
    df=df,
    year_cols=year_cols,
    country_col="Country",
    iso3_col="ISO3",
    tech_col="Technology_std",
    start_year=start,
    width=1200,
    height=660,
)



In [ ]:
import pandas as pd
from Isea import BubblePack

df = pd.read_csv("Isea/Energy_clean.csv")

# (opcional) deja sólo capacidad instalada en MW
df = df[(df["Indicator"] == "Electricity Installed Capacity") &
        (df["Unit"] == "Megawatt (MW)")].copy()

# Asegura Technology_std
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
if "Technology_std" not in df.columns:
    df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

# Elige año disponible
year_cols = [c for c in df.columns if isinstance(c, str) and c.startswith("F") and df[c].notna().any()]
YEAR = "F2023" if "F2023" in year_cols else year_cols[-1]

# OPCIÓN A: color por tecnología dominante (default)
BubblePack(
    df=df,
    year=YEAR,
    color_by="DominantTech",   # o "Region" si tienes esa columna
    value_mode="total",        # "total" o "renewables"
    width=1200,
    height=660
)

# OPCIÓN B (si tienes columna "Region" en tu df):
# BubblePack(df=df, year=YEAR, color_by="Region", value_mode="total")


In [ ]:
# Visualización de eficiencia de combustible por año
import pandas as pd
from Isea.graphs import RadialStackedBar

# 1. PREPARACIÓN DE DATOS
df = pd.read_csv("Isea/datos_vis.csv")
df = df.dropna(subset=["mpg", "horsepower"])

# 2. TRANSFORMACIÓN: Crear categorías de eficiencia similares a las de la imagen
df['efficiency_category'] = pd.cut(
    df['mpg'],
    bins=[0, 15, 20, 25, 30, 35, 100],
    labels=[
        'Muy bajo (<15 mpg)', 
        'Bajo (15-20)', 
        'Moderado (20-25)',
        'Eficiente (25-30)', 
        'Muy eficiente (30-35)', 
        'Ultra eficiente (>35)'
    ]
)

# Colores similares a la imagen (de rojo a azul)
custom_color_scheme = [
    '#d73027',  # Rojo
    '#fc8d59',  # Naranja
    '#fee08b',  # Amarillo
    '#d9ef8b',  # Verde claro
    '#91cf60',  # Verde
    '#1a9850'   # Verde oscuro
]

# 3. VISUALIZACIÓN MEJORADA
RadialStackedBar(
    df=df,
    group_col="model_year",
    category_col="efficiency_category", 
    value_col="horsepower",
    agg="mean",
    width=950,
    height=950,
    inner_radius=200,
    custom_colors=custom_color_scheme,  # Usar colores personalizados
    sort_on_click=True,
    title="Horsepower and Fuel Efficiency by Year"
)

In [ ]:
import pandas as pd
from Isea.graphs import RadialStackedBar

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]
for c in year_cols: df[c] = pd.to_numeric(df[c], errors="coerce")

IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
f = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()
YEAR = [c for c in year_cols if f[c].notna().any()][-1]

tech_map = {
"Hydropower (excl. Pumped Storage)": "Hydro",
"Solar energy": "Solar",
"Wind energy": "Wind",
"Bioenergy": "Bio",
"Fossil fuels": "Fossil",
}
f["Technology_std"] = f["Technology"].map(tech_map).fillna(f["Technology"])

countries_by_region = f.groupby("Region")["Country"].unique().apply(list)
selected_region = "Africa" # Puedes cambiarlo a: Europe, Americas, Asia, Oceania
countries = countries_by_region.get(selected_region, countries_by_region.iloc[0])

f_region = f[f["Country"].isin(countries[:15])].copy() # Tomar primeros 15 países

radial_data = f_region.pivot_table(
index="Country",
columns="Technology_std",
values=YEAR,
aggfunc="sum"
).fillna(0).reset_index()

RadialStackedBar(
df=radial_data,
group_col="Country",
category_col="Technology_std",
value_col=YEAR,
width=900,
height=900,
inner_radius=200,
color_scheme="schemeSpectral",
sort_on_click=True,
custom_colors=["#1a9850", "#91cf60", "#d9ef8b", "#fee08b", "#fc8d59", "#d73027"]
)